Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
#import dask.dataframe as dd
#!pip install fastparquet
#from fastparquet import ParquetFile

import re
import nltk
import json
import pandas as pd
import numpy as np
!pip install vaderSentiment
from nltk.corpus import wordnet 
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

nltk.download('wordnet')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_colwidth', 1000)

#imports to get our parquet files aand dask files

ModuleNotFoundError: ignored

In [ ]:
# Load pandas dataframe
def load_data(url):
    if url.endswith(".csv") or "csv" in url:
        df = pd.read_csv(url)
    else:
        df = None
    return df


In [ ]:
rev_url = 'https://tally-ai-dspt3.s3.amazonaws.com/yelp-restaurants/rev_export.csv'

rev = load_data(rev_url)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
rev.head()

,Unnamed: 0,review_id,reviews_stars,date,text
0,0,Oa-sapLub0wRAzxRKVmtDQ,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much fo..."
1,1,NawvdCkVIvhMO_eMVMdBhw,3.0,2010-12-04 00:38:28,There are several contributing factors to the ...
2,2,uBWpaM4_PQ1NMB3N9nYkJA,1.0,2012-03-07 21:51:40,Moving to the Phoenix area was a shock to begi...
3,3,EIcE_Xw9G5bDAY-XQ0Vosg,1.0,2011-10-06 00:29:51,Usually ok but today we went and ordered garde...
4,4,BXTxF_ijBmm5RPYELr-5RQ,4.0,2013-11-20 03:08:02,I have never dined in. The restaurant itself i...


In [ ]:
rev.columns

Index(['Unnamed: 0', 'review_id', 'reviews_stars', 'date', 'text'], dtype='object')

In [ ]:
rev = rev.drop(columns=["Unnamed: 0"])
rev.head()

,review_id,reviews_stars,date,text
0,Oa-sapLub0wRAzxRKVmtDQ,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much fo..."
1,NawvdCkVIvhMO_eMVMdBhw,3.0,2010-12-04 00:38:28,There are several contributing factors to the ...
2,uBWpaM4_PQ1NMB3N9nYkJA,1.0,2012-03-07 21:51:40,Moving to the Phoenix area was a shock to begi...
3,EIcE_Xw9G5bDAY-XQ0Vosg,1.0,2011-10-06 00:29:51,Usually ok but today we went and ordered garde...
4,BXTxF_ijBmm5RPYELr-5RQ,4.0,2013-11-20 03:08:02,I have never dined in. The restaurant itself i...


In [ ]:
rev.dtypes

review_id         object
reviews_stars    float64
date              object
text              object
dtype: object

In [ ]:
#causes errors when running sentiment analysis
rev['text'] = rev['text'].astype(str)

In [ ]:
rev['text'] = rev['text'].str.replace('\d+', '')

In [ ]:
rev['text'] = rev['text'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)

# Sentiment Analysis that worked on sample

In [ ]:
#load VADER
analyzer = SentimentIntensityAnalyzer()
#Add VADER metrics to dataframe



In [ ]:
rev['compound'] = [analyzer.polarity_scores(v)['compound'] for v in rev['text']]
rev['neg'] = [analyzer.polarity_scores(v)['neg'] for v in rev['text']]
rev['neu'] = [analyzer.polarity_scores(v)['neu'] for v in rev['text']]
rev['pos'] = [analyzer.polarity_scores(v)['pos'] for v in rev['text']]

rev.head(3)

KeyboardInterrupt: ignored

In [ ]:
rev['text_length'] = rev['text'].str.len()

In [ ]:
columns = ['review_id', 'business_id', 'compound']

In [ ]:
df = rev[columns]

In [ ]:
def tokenizer(doc):
     return [token for token in simple_preprocess(doc) 
             if token not in STOPWORDS]


def related_to_food(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('food.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list


def related_to_service(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('service.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list


def related_to_speed(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('speed.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list

def related_to_price(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('price.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list


def related_to_ambience(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('ambience.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list


def related_to_experience(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('experience.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list

def extract_subject_related_words():
  df['text'] = df['text'].apply(lambda x:" ".join(re.findall("[a-zA-Z]+", x)))
  df['cleaned'] = df['text'].apply(tokenizer)
  df['words_related_to_food'] = df['cleaned'].apply(related_to_food)
  df['words_related_to_service'] = df['cleaned'].apply(related_to_service)
  df['words_related_to_speed'] = df['cleaned'].apply(related_to_speed)
  df['words_related_to_price'] = df['cleaned'].apply(related_to_price)
  df['words_related_to_ambience'] = df['cleaned'].apply(related_to_ambience)
  df['words_related_to_experience'] = df['cleaned'].apply(related_to_experience)

extract_subject_related_words()

In [ ]:
def sentiment_score(sentence):
  # Create a SentimentIntensityAnalyzer object. 
  sid_obj = SentimentIntensityAnalyzer()
  # polarity_scores method of SentimentIntensityAnalyzer oject gives a sentiment dictionary. which contains pos, neg, neu, and compound scores. 
  sentiment_dict = sid_obj.polarity_scores(sentence)
  return sentiment_dict

def get_sentiment(review_list):
  all_sentiments = []
  compounds = []
  if len(review_list) > 0:
    for review in review_list:
      score = sentiment_score(review)
      all_sentiments.append(score)

  if len(all_sentiments) > 0:
    for sentiment_dict in all_sentiments:
      compound = sentiment_dict['compound']
      compounds.append(compound)

  if len(compounds) > 0:
    avg_sentiment = sum(compounds) / len(compounds)
	
  else:
    avg_sentiment = None
    
  return avg_sentiment

def get_subject_review_scores(review_list):
  if len(review_list) > 0:
    sub_sentiment_score = round((get_sentiment(review_list))*150)
    if sub_sentiment_score < 0:
      sub_sentiment_score = 0
    else:
      sub_sentiment_score = 75 #leaving at 75 to give neutral score
    return sub_sentiment_score

def sentimental_analysis(df):
  # Create a SentimentIntensityAnalyzer object. 
  sid_obj = SentimentIntensityAnalyzer()
  # extract_subject_related_words
  df['text'] = df['text'].apply(lambda x:" ".join(re.findall("[a-zA-Z]+", x)))
  df['cleaned'] = df['text'].apply(tokenizer)

  df['words_related_to_food'] = df['cleaned'].apply(related_to_food)
  df['food_review_list'] = df[df['words_related_to_food'].map(len) > 1]['text'].tolist()
  df['food_score'] = df['food_review_list'].apply(get_subject_review_scores)

  df['words_related_to_service'] = df['cleaned'].apply(related_to_service)
  df['service_review_list'] = df[df['words_related_to_service'].map(len) > 1]['text'].tolist()
  df['service_score'] = df['service_review_list'].apply(get_subject_review_scores)

  df['words_related_to_speed'] = df['cleaned'].apply(related_to_service)
  df['speed_review_list'] = df[df['words_related_to_speed'].map(len) > 1]['text'].tolist()
  df['speed_score'] = df['speed_review_list'].apply(get_subject_review_scores)

  df['words_related_to_price'] = df['cleaned'].apply(related_to_service)
  df['price_review_list'] = df[df['words_related_to_price'].map(len) > 1]['text'].tolist()
  df['price_score'] = df['price_review_list'].apply(get_subject_review_scores)

  df['words_related_to_ambience'] = df['cleaned'].apply(related_to_service)
  df['ambience_review_list'] = df[df['words_related_to_ambience'].map(len) > 1]['text'].tolist()
  df['ambience_score'] = df['ambience_review_list'].apply(get_subject_review_scores)

  df['words_related_to_experience'] = df['cleaned'].apply(related_to_service)
  df['experience_review_list'] = df[df['words_related_to_experience'].map(len) > 1]['text'].tolist()
  df['experience_score'] = df['experience_review_list'].apply(get_subject_review_scores)
  
  return df

df.head(3)

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
df.to_csv('sentiment.csv')
!cp sentiment.csv "drive/My Drive/"

In [ ]:
# def sentiment_score(sentence):
#   # Create a SentimentIntensityAnalyzer object. 
#   sid_obj = SentimentIntensityAnalyzer()
#   # polarity_scores method of SentimentIntensityAnalyzer oject gives a sentiment dictionary. which contains pos, neg, neu, and compound scores. 
#   sentiment_dict = sid_obj.polarity_scores(sentence)
#   return sentiment_dict

# def get_sentiment(review_list):
#   all_sentiments = []
#   compounds = []
#   if len(review_list) > 0:
#     for review in review_list:
#       score = sentiment_score(review)
#       all_sentiments.append(score)
#   if len(all_sentiments) > 0:
#     for sentiment_dict in all_sentiments:
#       compound = sentiment_dict['compound']
#       compounds.append(compound)
# 	if len(compounds) > 0:
# 		avg_sentiment = sum(compounds) / len(compounds)
# 	else:
# 		avg_sentiment = None
# 	return avg_sentiment

# def get_subject_review_scores(review_list):
#     if len(review_list) > 0:
# 	    sub_sentiment_score = round((get_sentiment(review_list))*150)
# 		if sub_sentiment_score < 0:
# 		    sub_sentiment_score = 0
# 		else: 
# 			sub_sentiment_score = 75 #leaving at 75 to give neutral score
#     return sub_sentiment_score

# def sentimental_analysis(df):
#   # Create a SentimentIntensityAnalyzer object. 
#   sid_obj = SentimentIntensityAnalyzer()
#   # extract_subject_related_words
#   df['text'] = df['text'].apply(lambda x:" ".join(re.findall("[a-zA-Z]+", x)))
#   df['cleaned'] = df['text'].apply(tokenizer)
#   df['words_related_to_food'] = df['cleaned'].apply(related_to_food)
#   df['food_review_list'] = df[df['words_related_to_food'].map(len) > 1]['text'].tolist()
#   df['food_score'] = df['food_review_list'].apply(get_subject_review_scores)
# 	df['words_related_to_service'] = df['cleaned'].apply(related_to_service)
#   df['service_review_list'] = df[df['words_related_to_service'].map(len) > 1]['text'].tolist()
#   df['service_score'] = df['service_review_list'].apply(get_subject_review_scores)
# 	df['words_related_to_speed'] = df['cleaned'].apply(related_to_service)
#   df['speed_review_list'] = df[df['words_related_to_speed'].map(len) > 1]['text'].tolist()
#   df['speed_score'] = df['speed_review_list'].apply(get_subject_review_scores)
# 	df['words_related_to_price'] = df['cleaned'].apply(related_to_service)
#   df['price_review_list'] = df[df['words_related_to_price'].map(len) > 1]['text'].tolist()
#   df['price_score'] = df['price_review_list'].apply(get_subject_review_scores)
# 	df['words_related_to_ambience'] = df['cleaned'].apply(related_to_service)
#   df['ambience_review_list'] = df[df['words_related_to_ambience'].map(len) > 1]['text'].tolist()
#   df['ambience_score'] = df['ambience_review_list'].apply(get_subject_review_scores)
# 	df['words_related_to_experience'] = df['cleaned'].apply(related_to_service)
#   df['experience_review_list'] = df[df['words_related_to_experience'].map(len) > 1]['text'].tolist()
#   df['experience_score'] = df['experience_review_list'].apply(get_subject_review_scores)
#   return df

In [ ]:
df.head(3)

In [ ]:
df.to_csv('sentiment.csv')
!cp df.csv "drive/My Drive/"